### Importing Required Libraries

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import random as rand
import pickle as pkl
import warnings

### Getting Data Ready


In [2]:
yt=pd.read_csv(r'C:\Users\Varishu Pant\Desktop\Praxis docs\Excel\YtDataOld.csv')  #Reading Data

In [3]:
yt.shape

(347623, 37)

In [12]:
rand.seed(0)                                                                          #Setting seed for reproducble results
train_indices=np.random.choice(yt.index, round(0.9*len(yt.index)), replace=False)     #Train-Test Split(90-10)
yt_train=yt.iloc[train_indices,:]
test_indices=[index for index in yt.index if index not in train_indices]
yt_test=yt.iloc[test_indices,:]

In [5]:
yt_train.shape

(312861, 37)

In [6]:
yt_train.sample(5)              #Checking out the variables

,Unnamed: 0,Unnamed: 0.1,V_id,caption,categoryId,channelId,commentCount,defaultAudioLanguage,definition,dimension,...,video_title_length,tags_in_title,No_of_tags,channel_title_length,channel_description_length,months_old,channel_months_old,day_uploaded,social_links,twitter_url
216511,216511,216512,YYeT9fOnu0o,0,8,UCVTyTA7-g9nopHeHbeuvpRA,94.0,NaN,0,0,...,13,17,32,5,50,11,44.0,0,5,https://twitter.com/LateNightSeth
253511,253511,253512,jS9Ep4g0jvI,0,10,UCXIJgqnII2ZOINSWNOGFThA,339.0,NaN,1,0,...,8,1,10,2,88,2,124.0,6,14,https://twitter.com/FoxNews
333332,333332,333333,qcXyez5Cg2U,0,14,UCY5dEGAac6AIIB-z5U5E-9w,26.0,NaN,1,0,...,4,3,8,2,31,48,85.0,4,8,http://twitter.com/canalmormon
171496,171496,171497,VvxkDAVvkMg,0,7,UCiuBAp4W5HIW7reX9_uxQ1A,3968.0,NaN,0,0,...,8,1,3,3,1,59,91.0,5,1,NaN
136544,136544,136544,3YkNp1KEnZ0,0,5,UC_FZP9M_5n7VDKoshyb5WYQ,4.0,NaN,0,0,...,9,0,0,4,51,12,55.0,2,4,NaN


In [14]:
null=(yt_train.isnull().sum()/yt_train.shape[0])*100  #Looking for missing values
null[null>0]

Series([], dtype: float64)

In [13]:
to_remove=['Unnamed: 0','Unnamed: 0.1','twitter_url','defaultAudioLanguage']  #Removing Unncessary Columns
yt_train=yt_train.drop(to_remove,axis=1)

Since the variables 'defaultAudioLanguage' has more than 80% values missing,we decide to remove it from the analysis.
Since twitter_url contains a lot of missing values and serves no relevance to the prediction problem,we remove it too.
Also,columns titled 'Unnamed: 0' and 'Unnamed: 0.1' are not required as they serve merely as primary ids.

In [16]:
yt_train.dtypes

V_id                           object
caption                         int64
categoryId                      int64
channelId                      object
commentCount                  float64
definition                      int64
dimension                       int64
dislikeCount                  float64
duration                      float64
embeddable                      int64
licencedContent                 int64
license                         int64
likeCount                     float64
privacyStatus                   int64
projection                      int64
thumbnail                      object
viewCount                       int64
channel_ViewCount               int64
channel_commentCount            int64
channel_subscriberCount         int64
channel_videoCount              int64
description_length              int64
http_in_descp                   int64
tags_in_desc                    int64
video_title_length              int64
tags_in_title                   int64
No_of_tags  

In [19]:
yt_train[['day_uploaded','categoryId']].sample(3)

,day_uploaded,categoryId
228253,1,9
61837,0,2
275003,4,11


In [20]:
warnings.filterwarnings('ignore')

weekdays=['Friday','Monday','Saturday','Sunday','Thursday','Tuesday','Wednesday']
categories=['Film & Animation','Autos & Vehicles','Music','Pets & Animals','Sports','Travel & Events','Gaming','People & Blogs','Comedy','Entertainment','News & Politics','Howto & Style','Education','Science & Technology','Nonprofits & Activism']

for i in range(0,7):
    yt_train.day_uploaded[yt_train.day_uploaded==i]=weekdays[i]
for i in range(0,15):
    yt_train.categoryId[yt_train.categoryId==i]=categories[i]

In [21]:
yt_train[['day_uploaded','categoryId']].sample(3)

,day_uploaded,categoryId
306711,Saturday,Science & Technology
265457,Monday,News & Politics
323101,Sunday,Science & Technology


### Creating Derived Variable: YearPublished


In [23]:
warnings.filterwarnings('ignore')

yt_train['YearPublished']=0
years=[2010,2011,2012,2013,2014,2015,2016]
for i in range(0,7):
    yt_train['YearPublished'][yt_train[(yt_train['months_old']>12*i) & (yt_train['months_old']<=12*(i+1))].index]=years[i]

In [24]:
yt_train['YearPublished'].value_counts()

2012    45635
2014    45616
2011    44866
2015    44814
2013    44808
2010    43735
2016    43383
0           4
Name: YearPublished, dtype: int64

In [26]:
yt_train[yt_train['YearPublished']==0]

,V_id,caption,categoryId,channelId,commentCount,definition,dimension,dislikeCount,duration,embeddable,...,video_title_length,tags_in_title,No_of_tags,channel_title_length,channel_description_length,months_old,channel_months_old,day_uploaded,social_links,YearPublished
115249,YwQJ4Txp0Lo,0,Sports,UCXrvAIp3vLwn0sGa4xf6Tyw,428.0,0,0,28.0,441.0,1,...,13,7,54,3,125,85,98.0,Thursday,5,0
231551,w0jy4-jrZhQ,0,Entertainment,UChPYHetKwsJ7c0Ye-Z-3jbQ,124.0,1,0,73.0,130.0,1,...,12,4,8,1,1,85,88.0,Thursday,1,0
345249,subfKHjBCXo,0,Nonprofits & Activism,UCcoAXDpnRim0Bl5d6keXapA,35.0,1,0,15.0,286.0,1,...,13,0,1,3,1,85,98.0,Thursday,1,0
343246,h1LNJsH-jDU,0,Nonprofits & Activism,UCTev5U7yYiN7-ZouIwAmebQ,48.0,1,0,16.0,1205.0,1,...,4,3,16,6,113,85,93.0,Wednesday,1,0


In [27]:
yt_train[yt_train['months_old']>84]

,V_id,caption,categoryId,channelId,commentCount,definition,dimension,dislikeCount,duration,embeddable,...,video_title_length,tags_in_title,No_of_tags,channel_title_length,channel_description_length,months_old,channel_months_old,day_uploaded,social_links,YearPublished
115249,YwQJ4Txp0Lo,0,Sports,UCXrvAIp3vLwn0sGa4xf6Tyw,428.0,0,0,28.0,441.0,1,...,13,7,54,3,125,85,98.0,Thursday,5,0
231551,w0jy4-jrZhQ,0,Entertainment,UChPYHetKwsJ7c0Ye-Z-3jbQ,124.0,1,0,73.0,130.0,1,...,12,4,8,1,1,85,88.0,Thursday,1,0
345249,subfKHjBCXo,0,Nonprofits & Activism,UCcoAXDpnRim0Bl5d6keXapA,35.0,1,0,15.0,286.0,1,...,13,0,1,3,1,85,98.0,Thursday,1,0
343246,h1LNJsH-jDU,0,Nonprofits & Activism,UCTev5U7yYiN7-ZouIwAmebQ,48.0,1,0,16.0,1205.0,1,...,4,3,16,6,113,85,93.0,Wednesday,1,0


4 videos are from 2009 so we remove them from the data

In [29]:
yt_train=yt_train.drop(index=yt_train[yt_train['YearPublished']==0].index,axis=0)
yt_train['YearPublished'].value_counts()

2012    45635
2014    45616
2011    44866
2015    44814
2013    44808
2010    43735
2016    43383
Name: YearPublished, dtype: int64

In [31]:
yt_train.duration.describe()

count    312857.000000
mean        493.850756
std         805.936636
min           0.000000
25%         184.000000
50%         281.000000
75%         514.000000
max       86401.000000
Name: duration, dtype: float64

In [35]:
durationNa=yt_train[yt_train['duration']==yt_train['duration'].min()] #Imputing 0 duration video
len(durationNa)

35

In [37]:
np.random.seed(10)
durationNa.sample(5)

,V_id,caption,categoryId,channelId,commentCount,definition,dimension,dislikeCount,duration,embeddable,...,video_title_length,tags_in_title,No_of_tags,channel_title_length,channel_description_length,months_old,channel_months_old,day_uploaded,social_links,YearPublished
237961,syo83iYZEAc,0,Entertainment,UCFMGtmqCPSbi6EZQufkPLeA,0.000000,1,0,6.0,0.0,1,...,9,0,0,1,59,11,59.0,Thursday,1,2010
305983,Xm1bHwiTwoA,0,Science & Technology,UCDPUOq2ADKZwRleMUImX-Ww,1902.863643,1,0,2.0,0.0,1,...,6,6,14,1,22,5,90.0,Tuesday,3,2010
296580,wYMs-2LQVcw,0,Education,UCivWIT8SpFaJh6Cz823GwXw,0.000000,1,0,3.0,0.0,1,...,5,9,61,3,9,6,7.0,Tuesday,5,2010
323223,SF7FUU7CThs,0,Science & Technology,UCIR_LPmEQ9QHR0yB2lxgaxQ,4.000000,1,0,2519.0,0.0,1,...,15,9,14,4,132,2,35.0,Thursday,4,2010
303064,_E7iZKi_J8g,0,Education,UCt0JCY2fgYNnQfKCA6LdZiQ,0.000000,1,0,140.0,0.0,1,...,5,4,17,3,4,2,7.0,Monday,0,2010


Videos were not on the platform anymore.Some were unavailable while others were removed.Some of them were livestreams that were no lonnger accesible.

We remove these videos

In [38]:
yt_train=yt_train.drop(index=yt_train[yt_train['duration']==0].index,axis=0)

In [40]:
yt_train.duration.describe()

count    312822.00000
mean        493.90601
std         805.96479
min           3.00000
25%         184.00000
50%         281.00000
75%         514.00000
max       86401.00000
Name: duration, dtype: float64

In [41]:
yt_train.license.value_counts()

1    310218
0      2604
Name: license, dtype: int64

In [42]:
yt_train.licencedContent.value_counts()

1    200133
0    112689
Name: licencedContent, dtype: int64

In [51]:
def pipeline(data):
    import pandas as pd
    import matplotlib.pyplot as plt
    import seaborn as sns
    import numpy as np
    import random as rand
    import pickle as pkl
    import warnings
    warnings.filterwarnings('ignore')
    to_remove=['Unnamed: 0','Unnamed: 0.1','twitter_url','defaultAudioLanguage']  #Removing Unncessary Columns
    data=data.drop(to_remove,axis=1)
   

    weekdays=['Friday','Monday','Saturday','Sunday','Thursday','Tuesday','Wednesday']
    categories=['Film & Animation','Autos & Vehicles','Music','Pets & Animals','Sports','Travel & Events','Gaming','People & Blogs','Comedy','Entertainment','News & Politics','Howto & Style','Education','Science & Technology','Nonprofits & Activism']

    for i in range(0,7):
        data.day_uploaded[data.day_uploaded==i]=weekdays[i]
    for i in range(0,15):
        data.categoryId[data.categoryId==i]=categories[i]

    data['YearPublished']=0
    years=[2010,2011,2012,2013,2014,2015,2016]
    for i in range(0,7):
        data['YearPublished'][data[(data['months_old']>12*i) & (data['months_old']<=12*(i+1))].index]=years[i]
    
    data=data.drop(index=data[data['YearPublished']==0].index,axis=0)
    
    data=data.drop(index=data[data['duration']==0].index,axis=0)
    return(data)


In [52]:
processed_test=pipeline(yt_test)